# Word Mover's Distance

In this notebook, we will see an application of Optimal Transport to the problem of computing similarities between sentences and texts. The method under the lens is called 'Word Mover's Distance' about 'Earth Mover's Distance', another name of the Wasserstein $1$ distance, mostly used in computer vision. 

Traditionally, portions of texts are compared by Cosine similarity on bag-of-words vectors, i.e. histograms of occurrences of words in a text. It captures the exact similarity in terms of words, but two very related sentences can be orthogonal if the words that are used have the same semantic but are different. Such a semantic distance can be obtained by using *word embeddings*, that are embeddings of words in a Euclidean space (of potentially large dimension) where the Euclidean distance has a semantic meaning: two related words will be close in such embeddings. A popular embedding is the *word2vec* embedding, obtained with neural networks. A study of those mechanisms is not in the scope of this notebook, but the interested reader can find more information on [the corresponding Wikipedia page](https://en.wikipedia.org/wiki/Word2vec). Throughout the rest of this tutorial, we will use a subset of the [GloVe] (https://nlp.stanford.edu/projects/glove/) embedding.

The key observation made by Kusner and colleagues [1] is that when confronted to a sentence/document, the optimal transport distance can be used between histograms of occurring words using a ground metric obtained through word embeddings. In such a way, related words will be matched together, and the resulting distance will somehow express semantic relatedness between the content.

[1] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International Conference on Machine Learning (pp. 957-966). http://proceedings.mlr.press/v37/kusnerb15.pdf

## A basic example 

We will start by reproducing Figure $1$ in the original paper:

<img src="https://remi.flamary.com/cours/otml/fig1.png">

Two sentences are considered: 'Obama speaks to the media in Illinois' and 'The president greets the press in Chicago'. It is clear from this example that the Cosine similarity between the two sentences indicates that the two sentences are not related since there is no word in common.  We will start by some imports and creating a list of the two sentences as words without stopwords that are not relevant for our analysis.

In [ ]:
import os
import numpy as np
import matplotlib.pylab as pl
import ot


s1 = ['Obama','speaks','media','Illinois']
s2 = ['President','greets','press','Chicago']

We will use a subset of the GloVe word embedding, expressed as a dictionary ```(word, embedding)``` that you can load this way:

In [ ]:
# Load dict   
model=dict(np.load('data/model.npz'))

Then the embedded representation of the sentences can be obtained by:

In [ ]:
s1_embed = np.array([model[w] for w in s1])
s2_embed = np.array([model[w] for w in s2])

From the multidimensional scaling method in ```sklearn```, try to visualize the corresponding embedding of words in 2D:

In [ ]:
from sklearn import manifold

C = ot.dist(np.vstack((s1_embed,s2_embed)))

nmds = manifold.MDS(
        2,
        eps=1e-9,
        dissimilarity="precomputed",
        n_init=1)
npos = nmds.fit_transform(C)

pl.figure(figsize=(6,6))
pl.scatter(npos[:4,0],npos[:4,1],c='r',s=50, edgecolor = 'k')
for i, txt in enumerate(s1):
    pl.annotate(txt, (npos[i,0]-4,npos[i,1]+2),fontsize=20)
pl.scatter(npos[4:,0],npos[4:,1],c='b',s=50, edgecolor = 'k')
for i, txt in enumerate(s2):
    pl.annotate(txt, (npos[i+4,0]-4,npos[i+4,1]+2),fontsize=20)
pl.axis('off')
pl.tight_layout()
pl.show()

Let's now compute the coupling between those two distributions and visualize the corresponding result:

In [ ]:
C2= ot.dist(s1_embed,s2_embed)
G=ot.emd(ot.unif(4),ot.unif(4),C2)

pl.figure(figsize=(6,6))
pl.scatter(npos[:4,0],npos[:4,1],c='r',s=50, edgecolor = 'k')
for i, txt in enumerate(s1):
    pl.annotate(txt, (npos[i,0]-4,npos[i,1]+2),fontsize=20)
pl.scatter(npos[4:,0],npos[4:,1],c='b',s=50, edgecolor = 'k')
for i, txt in enumerate(s2):
    pl.annotate(txt, (npos[i+4,0]-4,npos[i+4,1]+2),fontsize=20)
for i in range(G.shape[0]):
    for j in range(G.shape[1]):
        if G[i,j]>1e-5:
            pl.plot([npos[i,0],npos[j+4,0]],[npos[i,1],npos[j+4,1]],'k',alpha=G[i,j]/np.max(G))
pl.title('Word embedding and coupling with OT')
pl.axis('off')
pl.tight_layout()
pl.show()

## Sentence similarity
We will now explore the superiority of this Word mover distance (WMD) in a regression context, where our goal is to estimate the similarity (or relatedness) of two sentences on a scale of 0 to 5 (5 being the most similar). Given a set of pairs of sentences with a human-annotated relatedness, our goal is to predict the relatedness from a new pair of sentences.

We will use the [SICK (Sentences Involving Compositional Knowledge) dataset](http://clic.cimec.unitn.it/composes/sick.html) for this purpose.

We first load it:

In [ ]:
# Load data
data=np.load('data/data_text.npz')   
setA=data['setA']
setB=data['setB']
scores=data['scores']

# Some important prints
print (setA[0])
print (setB[0])
print(scores[0])

# Save data for later usage
np.savez('data/data_text.npz',setA=setA,setB=setB,scores=scores)

We will only keep 200 sentences for learning our regression model and the rest for testing:

In [ ]:
n=200
testA=setA[n:]
trainA=setA[:n]
testB=setB[n:]
trainB=setB[:n]

scores_train=scores[:n]
scores_test=scores[n:]

Using the countVectorizer model from ```sklearn```, compute all the bag-of-words representations of the sentences:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words="english").fit(np.hstack((setA,setB)))

Build a big data matrix of all the words present in the dataset embeddings:

In [ ]:
all_feat = [model[w] for w in vect.get_feature_names()]
all_feat = np.array(all_feat)

Compute a big matrix of all pairwise feature distances using the ```dist()``` method of ```POT```:

In [ ]:
D = ot.dist(all_feat)

Now you can write a code that will compute the Cosine and WMD dissimilarities from all the pairs of the training set:

In [ ]:
# Create lists to append data
X_cos=[]
X_wmd=[]
Y=[]

# Append data to the lists
for i in range(len(trainA)):
    s1 = vect.transform([trainA[i]]).toarray().ravel()
    s2 = vect.transform([trainB[i]]).toarray().ravel()
    # cosine similarity between bag of words
    d_cos=np.cos(np.dot(s1,s2))
    X_cos.append(d_cos)
    # WMD
    s1=s1.astype(np.float)/np.sum(s1)
    s2=s2.astype(np.float)/np.sum(s2)
    d_wmd=ot.emd2(s1,s2,D)
    X_wmd.append(d_wmd)
    Y.append(scores_train[i])

Visualize the corresponding golden similarities/distance from the learning set. Hence you have a first appreciation of how much WMD better captures this similarity.

In [ ]:
pl.figure()
pl.scatter(X_cos,Y)
pl.title('Cosine Similarity VS golden score')
pl.show()
pl.figure()
pl.scatter(X_wmd,Y)
pl.title('WMD Similarity VS golden score')
pl.show()

You can learn a simple regression model between those 2 quantities. Use a polynomial of degree 2 to learn the regression model.

In [ ]:
import numpy.polynomial.polynomial as poly
k_cos = poly.polyfit(X_cos, Y, 2)
k_wmd = poly.polyfit(X_wmd, Y, 2)

Now compute from your regression model the estimated relatedness for all the pairs in the test set.

In [ ]:
X_cos=[]
X_wmd=[]
Y_test=[]
for i in range(len(testA)):
    s1 = vect.transform([testA[i]]).toarray().ravel()
    s2 = vect.transform([testB[i]]).toarray().ravel()
    # cosine similarity between bag of words
    d_cos=np.cos(np.dot(s1,s2))
    X_cos.append(d_cos)
    # WMD
    s1=s1.astype(np.float)/np.sum(s1)
    s2=s2.astype(np.float)/np.sum(s2)
    d_wmd=ot.emd2(s1,s2,D)
    X_wmd.append(d_wmd)
    Y_test.append(scores_test[i])

Y_cos = poly.polyval(X_cos, k_cos)
Y_wmd = poly.polyval(X_wmd, k_wmd)

We will use MSE, Spearman's $\rho$ and Pearson coefficients to measure the quality of our regression model.

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import pearsonr
from scipy.stats import spearmanr

Estimate the quality of your regression model for both Cosine and WMD dissimilarities:

In [ ]:
print('-------- Cosine')

pr = pearsonr(Y_cos, Y_test)[0]
sr = spearmanr(Y_cos,Y_test)[0]
se = mse(Y_cos,Y_test)

print('Test Pearson (test): ' + str(pr))
print('Test Spearman (test): ' + str(sr))
print('Test MSE (test): ' + str(se))

print('-------- WMD')

pr = pearsonr(Y_wmd, Y_test)[0]
sr = spearmanr(Y_wmd,Y_test)[0]
se = mse(Y_wmd,Y_test)

print('Test Pearson (test): ' + str(pr))
print('Test Spearman (test): ' + str(sr))
print('Test MSE (test): ' + str(se))

Not bad isn't it?